In [1]:
from geopy.distance import geodesic

import pandas as pd

In [2]:
# load your data
df = pd.read_csv("states_centers.csv")

# build dict: key = ST2, value = (LATITUDE, LONGITUDE)
state_coords = dict(zip(df["ST2"], zip(df["LATITUDE"], df["LONGITUDE"])))

# example usage
print(state_coords["CA"])  # -> (35.491035, -119.347852)


df2 = pd.read_excel("tabelas.xlsx", sheet_name="table1")
df3 = pd.read_excel("tabelas.xlsx", sheet_name="tableb")

warehouse_coords = dict(
    zip(df2["Location (city)"], zip(df2["Latitude"], df2["Longitude"]))
)

print(warehouse_coords["Chicago"])  # -> (41.8781, -87.6298)

plants_coords = dict(zip(df3["Plant"], zip(df3["Latitude"], df3["Longitude"])))
print(plants_coords["New York, NY"])  # -> (40.7128, -74.006)

(35.491035, -119.347852)
(41.8781, -87.6298)
(40.7128, -74.006)


In [3]:
df_plants = pd.DataFrame(
    index=warehouse_coords.keys(), columns=plants_coords.keys(), dtype=float
)
for plant, p_coord in plants_coords.items():
    for wh, w_coord in warehouse_coords.items():
        df_plants.at[wh, plant] = geodesic(p_coord, w_coord).miles
df_plants.index.name = "Warehouse Location"

In [4]:
df_states = pd.DataFrame(
    index=state_coords.keys(), columns=warehouse_coords.keys(), dtype=float
)
for state, s_coord in state_coords.items():
    for wh, w_coord in warehouse_coords.items():
        df_states.at[wh, state] = geodesic(s_coord, w_coord).miles
df_states.index.name = "Warehouse Location"

In [5]:
df_p_d = pd.DataFrame(
    index=state_coords.keys(), columns=plants_coords.keys(), dtype=float
)
for plant, p_coord in plants_coords.items():
    for state, s_coord in state_coords.items():
        df_p_d.at[state, plant] = geodesic(p_coord, s_coord).miles
df_p_d.index.name = "State"

In [6]:
with pd.ExcelWriter("distance_matrix.xlsx", engine="openpyxl") as writer:
    df_plants.to_excel(writer, sheet_name="d_pw")
    df_states.to_excel(writer, sheet_name="d_ws")
    df_p_d.to_excel(writer, sheet_name="d_pd")